In [9]:
## packages for data table processing 
library(tidyverse)
library(stringr)
library(RColorBrewer)
library(data.table)
library(here)

In [8]:
library(here)
library(tidyverse)
library(data.table)


In [18]:
ss <- function(x, pattern, slot = 1, ...) { 
  sapply(strsplit(x = x, split = pattern, ...), '[', slot) }
options(stringsAsFactors = F)
options(repr.plot.width=11, repr.plot.height=8.5)

FIGDIR='figures/exploratory/differential_expression'
dir.create(here(FIGDIR, 'plots', 'gsea'), recursive = T, showWarnings = F)
dir.create(here(FIGDIR, 'tables'), recursive = T, showWarnings = F)

In [2]:
# 1) load in the DEG table from the big analyses
save_res_fn = here('data/tidy_data/differential_expression_analysis', 'rdas', 'OUD_Striatum_voom_limma_bigModelSVA_N22.celltype.rds')
res = readRDS(save_res_fn) %>% 
  lapply(function(x) x %>% filter(adj.P.Val.Between < alpha)) %>% 
  rbindlist() %>% filter(celltype %in% glia_types) %>% 
  arrange(adj.P.Val.Between)

Warning message in gzfile(file, "rb"):
“cannot open compressed file '/projects/pfenninggroup/singleCell/Pierce_transgen_rat_multiome/data/tidy_data/differential_expression_analysis/rdas/OUD_Striatum_voom_limma_bigModelSVA_N22.celltype.rds', probable reason 'No such file or directory'”


ERROR: Error in gzfile(file, "rb"): cannot open the connection


In [10]:
ss <- function(x, pattern, slot = 1, ...) { 
  sapply(strsplit(x = x, split = pattern, ...), '[', slot) }
options(stringsAsFactors = F)
options(repr.plot.width=11, repr.plot.height=8.5)


In [12]:
# 1) read in the DEG lists and rank order the genes by each differential comparisons
df_wide = readRDS(here('data/tidy_data/differential_expression/rdas/diff_gene_3-way_countsplit.rds'))

metric_match = "p_val_|avg_log2FC_|p_val_adj_|p_val_bonf_"
group_match = "_Met_vs_Sal|_Met_vs_Coc|_Coc_vs_Sal"

In [13]:
## grab the p-values and the log2FC differences
df_long = df_wide %>% pivot_longer(-c(gene:AveExpr)) %>% 
  filter(!grepl('effect|inter',name)) %>% 
  mutate(group = gsub(metric_match, '', name), 
         metric = gsub(group_match, '', name)) %>% 
  dplyr::select(-name) %>% 
  pivot_wider(names_from = 'metric') 

In [14]:

## differentially expressed genes
alpha = 0.001
df_long = df_long %>% filter(p_val_adj < alpha) %>% arrange(p_val)
table( df_long$celltype, df_long$group)

                  
                   Coc_vs_Sal Met_vs_Coc Met_vs_Sal
  Astrocytes              486        450        493
  D1-ICj-MSN              623        700        590
  D1-MSN                 1309        715        845
  D1-NUDAP-MSN            596        560        580
  D1/D2-Hybrid-MSN        689        657        670
  D1/D3-MSN               772        693        712
  D2-MSN                 1105        705        967
  Microglia               433        429        437
  Oligos                  449        453        452
  Oligos_Pre              577        456        487
  Pvalb-INT               688        682        684
  Sst-INT                 672        674        673

In [19]:
# 2) read in the clustered pathway enrichment tables
curenrich_clustered = 
  here(FIGDIR, "tables","clustered_gsea_pathway_network_interaction_sets.xlsx") %>% 
  readxl::read_xlsx() %>% mutate(
    leadingEdge = str_split(leadingEdge, ',')
  ) %>% unnest(leadingEdge) %>% dplyr::rename('gene'= 'leadingEdge') %>% 
  dplyr::rename('group' = 'comparison')

In [20]:
## D1H and Microglia Synapse cluster
curenrich_clustered %>%
  filter(cluster_number %in% c(1)) %>%
  count(gene, celltype, group) %>% 
  arrange(gene) %>%
  inner_join(df_long) %>% 
  filter(n > 1) %>% 
  as.data.frame()

Joining, by = c("gene", "celltype", "group")


gene,celltype,group,n,AveExpr,p_val,avg_log2FC,p_val_adj,p_val_bonf
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Adcy5,Microglia,Coc_vs_Sal,3,1.07619048,5.696252e-04,-1.2881383,0.0000000000,1
Akap5,Microglia,Coc_vs_Sal,3,0.03333333,2.116312e-06,-0.7068116,0.0006585595,1
Cacna1e,Microglia,Coc_vs_Sal,2,1.29761905,3.113199e-03,-0.9940258,0.0000000000,1
Cacna2d3,Microglia,Coc_vs_Sal,2,2.96428571,1.019331e-03,-1.4421130,0.0000000000,1
Cacnb2,Microglia,Coc_vs_Sal,2,1.52857143,1.020091e-03,-1.4204156,0.0000000000,1
Cacnb4,Microglia,Coc_vs_Sal,2,1.21428571,5.313095e-04,-1.4795746,0.0000000000,1
Camk4,Microglia,Coc_vs_Sal,3,0.60952381,1.632351e-03,-0.8609989,0.0000000000,1
Dlg2,Microglia,Coc_vs_Sal,3,6.63571429,6.305234e-03,-0.7964298,0.0000000000,1
Erbb4,Microglia,Coc_vs_Sal,3,1.75238095,7.933624e-02,-0.7409057,0.0000000000,1


In [23]:
## add description
curenrich_clustered %>%
  filter(cluster_number %in% c(2)) %>%
  count(gene, celltype, group) %>% 
  arrange(gene) %>%
  inner_join(df_long) %>% 
  filter(n > 1) %>% 
  as.data.frame()


Joining, by = c("gene", "celltype", "group")


gene,celltype,group,n,AveExpr,p_val,avg_log2FC,p_val_adj,p_val_bonf
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Abhd12,Oligos_Pre,Coc_vs_Sal,3,0.13636364,4.099472e-06,0.32672012,7.838787e-04,1.000000e+00
Akap6,Oligos_Pre,Coc_vs_Sal,6,0.82015810,1.070523e-05,0.27020157,0.000000e+00,1.000000e+00
Ank2,Oligos_Pre,Coc_vs_Sal,3,2.55204216,2.190000e-03,0.08357655,0.000000e+00,1.000000e+00
Ank3,Oligos_Pre,Coc_vs_Sal,3,2.32872200,5.524073e-04,0.15717041,0.000000e+00,1.000000e+00
Ano4,Oligos_Pre,Coc_vs_Sal,2,0.70750988,6.071939e-04,0.22318099,0.000000e+00,1.000000e+00
Atp1a3,Oligos_Pre,Coc_vs_Sal,3,0.03162055,2.799295e-06,0.33756327,8.138591e-04,1.000000e+00
Cacna1c,Microglia,Coc_vs_Sal,3,1.16190476,3.601903e-03,-0.90676418,0.000000e+00,1.000000e+00
Cacna1c,Oligos_Pre,Coc_vs_Sal,7,0.95454545,2.897307e-08,0.47702930,0.000000e+00,2.038499e-02
Cacna1d,Microglia,Coc_vs_Sal,3,0.66428571,5.914043e-02,-0.86946556,0.000000e+00,1.000000e+00


In [24]:
curenrich_clustered %>%
  filter(cluster_number %in% c(3)) %>%
  count(gene, celltype, group) %>% 
  arrange(gene) %>%
  inner_join(df_long) %>% 
  filter(n > 1) %>% 
  as.data.frame()

Joining, by = c("gene", "celltype", "group")


gene,celltype,group,n,AveExpr,p_val,avg_log2FC,p_val_adj,p_val_bonf
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Alcam,D1/D3-MSN,Coc_vs_Sal,2,2.53264331,4.132060e-04,0.27866724,0.000000e+00,1.000000e+00
App,D1/D3-MSN,Coc_vs_Sal,2,0.56369427,3.690400e-03,0.13221749,0.000000e+00,1.000000e+00
Auts2,D1/D3-MSN,Coc_vs_Sal,3,6.65207006,5.174455e-09,0.25252895,0.000000e+00,3.640664e-03
Cdh4,D1/D3-MSN,Coc_vs_Sal,2,1.18232484,4.099244e-08,0.59417477,0.000000e+00,2.884162e-02
Cdkl5,D1/D3-MSN,Coc_vs_Sal,3,0.81369427,7.395236e-06,0.35477922,0.000000e+00,1.000000e+00
Ctnna2,D1/D3-MSN,Coc_vs_Sal,2,2.47611465,1.093036e-02,0.11550500,0.000000e+00,1.000000e+00
Dab1,D1/D3-MSN,Coc_vs_Sal,4,1.92834395,6.585804e-13,1.26072778,0.000000e+00,4.633666e-07
Dclk1,D1/D3-MSN,Coc_vs_Sal,4,1.58280255,6.199795e-04,0.26025846,0.000000e+00,1.000000e+00
Epha5,D1/D3-MSN,Coc_vs_Sal,2,0.72691083,7.429280e-03,0.10300685,0.000000e+00,1.000000e+00
